In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
path = 'data/'

# Meat Consumption

https://data.oecd.org/agroutput/meat-consumption.htm

In [3]:
meat_consumption_path = path + 'DP_LIVE_23112019095726322.csv'
meat_consumption = pd.read_csv(meat_consumption_path)

In [4]:
# Remove useless columns
#  - column INDICATOR contains only 'MEATCONSUMP', so we can remove it
#  - column Flag Codes contains only NaN, so we can remove it
#  - column FREQUENCY contains only A's
meat_consumption = meat_consumption.drop(columns = ['INDICATOR', 'Flag Codes', 'FREQUENCY'])

In [5]:
# MEASURE are in ['KG_CAP', 'THND_TONNE']. 
# We decided to take THND_TONNE:
# Thousand tonnes of carcass weight
meat_consumption = meat_consumption[meat_consumption['MEASURE'] == 'THND_TONNE']

In [6]:
meat_consumption = meat_consumption[ (meat_consumption['LOCATION'] == 'EU27') | (meat_consumption['LOCATION'] == 'GBR')]

In [7]:
# take between 2004 and 2019
meat_consumption = meat_consumption[ (meat_consumption['TIME'] >= 2004) & (meat_consumption['TIME'] <= 2019)]

In [8]:
# need to merge GBR and EU27 values to obtain 'EU28' values
meat_consumption = meat_consumption.groupby(['TIME', 'SUBJECT'])['Value'].sum().unstack(level=1)#['Value']

In [9]:
meat_consumption.head()

SUBJECT,BEEF,PIG,POULTRY,SHEEP
TIME,,,,
2004,8511.407567,20593.146591,10711.840027,1425.017246
2005,8523.961069,20641.388301,10986.219492,1437.810065
2006,8606.549289,20843.812041,10709.619355,1417.412487
2007,8664.085754,21661.708968,11148.525044,1394.124228
2008,8350.813777,21102.870919,11332.439203,1309.447936


In order to get the overall consumption of meat (all meats mixed), we need to sum their quantities. 
Meanwhile, we need to take care about a fact:

["Carcass weight to retail weight conversion factors are: 0.7 for beef and veal, 0.78 for pigmeat, and 0.88 for both sheep meat and poultry meat."](https://data.oecd.org/agroutput/meat-consumption.htm)

In [10]:

meat_consumption['all_consumption'] = ( 
                                    (0.7 * meat_consumption['BEEF']) +
                                    (0.78 * meat_consumption['PIG']) +
                                    (0.88 * (meat_consumption['SHEEP'] + meat_consumption['POULTRY'])))

In [11]:
# adding the year column for the plot
meat_consumption['years'] = meat_consumption.index

In [12]:
meat_consumption.head()

SUBJECT,BEEF,PIG,POULTRY,SHEEP,all_consumption,years
TIME,,,,,,
2004,8511.407567,20593.146591,10711.840027,1425.017246,32701.074038,2004
2005,8523.961069,20641.388301,10986.219492,1437.810065,33000.201633,2005
2006,8606.549289,20843.812041,10709.619355,1417.412487,32954.545916,2006
2007,8664.085754,21661.708968,11148.525044,1394.124228,33998.524382,2007
2008,8350.813777,21102.870919,11332.439203,1309.447936,33430.669644,2008


## All consumption graph

In [16]:
fig = px.line(meat_consumption, x='years', y='all_consumption')
fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='Meat consumption in Thousand tonnes of carcass weight')
fig.update_layout(title_text = 'Meat consumption of the 28 EU countries over the years',)
fig.show()